## Data Wrangling:
### 1. Web Scraping
### 2. Pandas Basics

## 1. Web Scraping

### 1.1 HTML example

In [ ]:
import urllib.request
import numpy as np
from requests import get
from time import sleep
import random

In [ ]:
url = 'https://www.wikipedia.org'
with urllib.request.urlopen(url) as response:
    html = response.read()

In [ ]:
html

### 1.2 IMBD extraction with BeautifulSoup

In [ ]:
from bs4 import BeautifulSoup


In [ ]:
headers = {'Accept-Language': 'en-US,en;q=0.8'} # If this is not specified, the default language is E  
#get request for sci-fi
response = get("https://www.imdb.com/search/title?genres=sci-fi&start=1&explore=title_type,genres&ref_=adv_prv",
               headers=headers)
    


In [ ]:
response = get("https://www.imdb.com/search/title?genres=sci-fi&start=1&explore=title_type,genres&ref_=adv_prv",
         )

In [ ]:
page_html = BeautifulSoup(response.text, 'html.parser')

In [ ]:
movie_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

In [ ]:
movie_containers

In [ ]:
titles = [] # for titles appending

In [ ]:
for container in movie_containers:

    #conditional for all with metascore
    if container.find('div', class_ = 'ratings-metascore') is not None:

        #title
        title = container.h3.a.text
        titles.append(title)

In [ ]:
titles

## 2. Pandas Basics


### 2.1  Series

In [ ]:
import pandas as pd
import numpy as np

#### A Series is a one-dimensional array-like object containing a sequence of values (of similar types to NumPy types) of the same type and an associated array of data labels, called its index.

In [ ]:
obj = pd.Series([4, 7, -5, 3])
obj

In [ ]:
obj.array

Since we did not specify an index for the data, a default one consisting of the integers 0 through N - 1

In [ ]:
obj.index

In [ ]:
obj2 = pd.Series([4, 7, -5, 3], index=["d", "b", "a", "c"])
obj2

In [ ]:
obj2.index

You can use labels in the index when selecting single values or a set of values:

In [ ]:
obj2["a"]

In [ ]:
obj2["d"] = 6
obj2[["c", "a", "d"]]

Using NumPy functions or NumPy-like operations, such as filtering with a Boolean array, scalar multiplication, or applying math functions, will preserve the index-value link:

In [ ]:
obj2[obj2 > 0]

In [ ]:
obj2 * 2

In [ ]:
import numpy as np
np.exp(obj2)

Another way to think about a Series is as a fixed-length, ordered dictionary, as it is a mapping of index values to data values. It can be used in many contexts where you might use a dictionary:

In [ ]:
"e" in obj2

In [ ]:
"b" in obj2

You can create a Series from a dictionary:



In [ ]:
sdata = {"Ohio": 35000, "Texas": 71000, "Oregon": 16000, "Utah": 5000}
obj3 = pd.Series(sdata)
obj3

In [ ]:
obj3.to_dict()

When you are only passing a dictionary, the index in the resulting Series will respect the order of the keys according to the dictionary’s keys method, which depends on the key insertion order. You can override this by passing an index with the dictionary keys in the order you want them to appear in the resulting Series:

In [ ]:
states = ["California", "Ohio", "Oregon", "Texas"]
obj4 = pd.Series(sdata, index=states)
obj4

 The isna and notna functions in pandas should be used to detect missing data:

In [ ]:
pd.isna(obj4)

In [ ]:
pd.notna(obj4)

In [ ]:
obj4.isna()

A useful Series feature for many applications is that it automatically aligns by index label in arithmetic operations:

In [ ]:
obj3

In [ ]:
obj4

In [ ]:
obj3 + obj4

Both the Series object itself and its index have a name attribute, which integrates with other areas of pandas functionality:

In [ ]:
obj4.name = "population"
obj4.index.name = "state"
obj4

In [ ]:
obj

In [ ]:
obj.index = ["Bob", "Steve", "Jeff", "Ryan"]
obj

### 2.2 DataFrames

#### A DataFrame represents a rectangular table of data and contains an ordered, named collection of columns, each of which can be a different value type (numeric, string, Boolean, etc.). The DataFrame has both a row and column index; it can be thought of as a dictionary of Series all sharing the same index.

There are many ways to construct a DataFrame, though one of the most common is from a dictionary of equal-length lists or NumPy arrays:

In [ ]:
data = {"state": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada", "Nevada"],
        "year": [2000, 2001, 2002, 2001, 2002, 2003],
        "pop": [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
df = pd.DataFrame(data)

In [ ]:
df

You can use `head` and `tail` for selection of first and last rows:

In [ ]:
df.head()

In [ ]:
df.tail()

If you specify a sequence of columns, the DataFrame’s columns will be arranged in that order. Notice the missing values in 'debt':

In [ ]:
df2 = pd.DataFrame(data, columns=["year", "state", "pop", "debt"])
df2

In [ ]:
df2.columns

A column in a DataFrame can be retrieved as a Series either by dictionary-like notation or by using the dot attribute notation:



In [ ]:
df2["state"]

In [ ]:
df2.year

Rows can also be retrieved by position or name with the special iloc and loc attributes (more on this later in “Selection on DataFrame with loc and iloc”):

In [ ]:
df2.loc[1]

In [ ]:
df2.iloc[2]

Columns can be modified by assignment. For example, the empty debt column could be assigned a scalar value or an array of values:

In [ ]:
df2["debt"] = 16.5
df2

In [ ]:
df2["debt"] = np.arange(6.)
df2

When you are assigning lists or arrays to a column, the value’s length must match the length of the DataFrame. If you assign a Series, its labels will be realigned exactly to the DataFrame’s index, inserting missing values in any index values not present:

In [ ]:
val = pd.Series([-1.2, -1.5, -1.7], index=["two", "four", "five"])
df2["debt"] = val
df2

Assigning a column that doesn’t exist will create a new column.

The del keyword will delete columns like with a dictionary. As an example, I first add a new column of Boolean values where the state column equals "Ohio":

In [ ]:
df2["eastern"] = df2["state"] == "Ohio"
df2

The del method can then be used to remove this column:

In [ ]:
del df2["eastern"]
df2.columns

Another common form of data is a nested dictionary of dictionaries:



In [ ]:
populations = {"Ohio": {2000: 1.5, 2001: 1.7, 2002: 3.6},
               "Nevada": {2001: 2.4, 2002: 2.9}}

If the nested dictionary is passed to the DataFrame, pandas will interpret the outer dictionary keys as the columns, and the inner keys as the row indices:

In [ ]:
df3 = pd.DataFrame(populations)
df3

You can transpose the DataFrame (swap rows and columns) with similar syntax to a NumPy array:



In [ ]:
df3.T

If a DataFrame’s index and columns have their name attributes set, these will also be displayed:

In [ ]:
df3

In [ ]:
df3.index.name = "year"
df3.columns.name = "state"
df3

DataFrame’s to_numpy method returns the data contained in the DataFrame as a two-dimensional ndarray:

In [ ]:
df3.to_numpy()

In [ ]:
df2.to_numpy()

#### Reindexing
Reindex means to create a new object with the values rearranged to align with the new index:



In [ ]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=["d", "b", "a", "c"])
obj

In [ ]:
obj2 = obj.reindex(["a", "b", "c", "d", "e"])
obj2

For ordered data like time series, you may want to do some interpolation or filling of values when reindexing. The method option allows us to do this, using a method such as `ffill`, which forward-fills the values:



In [ ]:
obj3 = pd.Series(["blue", "purple", "yellow"], index=[0, 2, 4])
obj3

In [ ]:
obj3.reindex(np.arange(6), method="ffill")

With DataFrame, reindex can alter the (row) index, columns, or both. When passed only a sequence, it reindexes the rows in the result:

In [ ]:
df = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=["a", "c", "d"],
                     columns=["Ohio", "Texas", "California"])
df

In [ ]:
df2 = df.reindex(index=["a", "b", "c", "d"])
df2

The columns can be reindexed with the columns keyword:



In [ ]:
states = ["Texas", "Utah", "California"]
df.reindex(columns=states)

#### Dropping entries
Dropping one or more entries from an axis is simple if you already have an index array or list without those entries, since you can use the reindex method or .loc-based indexing.

In [ ]:
obj = pd.Series(np.arange(5.), index=["a", "b", "c", "d", "e"])
obj

In [ ]:
new_obj = obj.drop("c")
new_obj

In [ ]:
obj.drop(["d", "c"])

In [ ]:
obj

With DataFrame, index values can be deleted from either axis. To illustrate this, we first create an example DataFrame:

In [ ]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=["Ohio", "Colorado", "Utah", "New York"],
                    columns=["one", "two", "three", "four"])
data

Calling drop with a sequence of labels will drop values from the row labels (axis 0):



In [ ]:
data.drop(index=["Colorado", "Ohio"])

To drop labels from the columns, instead use the columns keyword:

In [ ]:
data.drop(columns=["two"])

You can also drop values from the columns by passing axis=1 (which is like NumPy) or axis="columns":

In [ ]:
data.drop("two", axis=1)

In [ ]:
data.drop(["two", "four"], axis="columns")

#### Indexing, Selection, and Filtering
Series indexing works analogously to NumPy array indexing, except you can use the Series’s index values instead of only integers:

In [ ]:
obj = pd.Series(np.arange(4.), index=["a", "b", "c", "d"])
obj

In [ ]:
obj["b"]

In [ ]:
obj[1]

In [ ]:
obj[2:4]

In [ ]:
obj[["b", "a", "d"]]

In [ ]:
obj[[1, 3]]

In [ ]:
obj[obj < 2]

While you can select data by label this way, the preferred way to select index values is with the special loc operator:



In [ ]:
obj.loc[["b", "a", "d"]]

The reason to prefer loc is because of the different treatment of integers when indexing with `[]`. Regular `[]`-based indexing will treat integers as labels if the index contains integers, so the behavior differs depending on the data type of the index. For example:

In [ ]:
obj1 = pd.Series([1, 2, 3], index=[2, 0, 1])
obj2 = pd.Series([1, 2, 3], index=["a", "b", "c"])

In [ ]:
obj1

In [ ]:
obj2

In [ ]:
obj1[[0, 1, 2]]

In [ ]:
obj2[[0, 1, 2]]

When using `loc`, the expression `obj.loc[[0, 1, 2]]` will fail when the index does not contain integers:

In [ ]:
obj2.loc[[0, 1]]

Since loc operator indexes exclusively with labels, there is also an iloc operator that indexes exclusively with integers to work consistently whether or not the index contains integers:

In [ ]:
obj1.iloc[[0, 1, 2]]


In [ ]:
obj2.iloc[[0, 1, 2]]


#### Selection on DataFrame with loc and iloc
Like Series, DataFrame has special attributes `loc` and `iloc` for label-based and integer-based indexing, respectively. Since DataFrame is two-dimensional, you can select a subset of the rows and columns with NumPy-like notation using either axis labels `(loc)` or integers `(iloc)`.

Let’s select a single row by label:

In [ ]:
data.loc["Colorado"]

The result of selecting a single row is a Series with an index that contains the DataFrame’s column labels. To select multiple roles, creating a new DataFrame, pass a sequence of labels:

In [ ]:
data.loc[["Colorado", "New York"]]

You can combine both row and column selection in loc by separating the selections with a comma:

In [ ]:
data.loc["Colorado", ["two", "three"]]

We’ll then perform some similar selections with integers using iloc:

In [ ]:
data.iloc[2]

In [ ]:
data.iloc[[2, 1]]

In [ ]:
data.iloc[2, [3, 0, 1]]

In [ ]:
data.iloc[[1, 2], [3, 0, 1]]

Both indexing functions work with slices in addition to single labels or lists of labels:

In [ ]:
data.loc[:"Utah", "two"]

In [ ]:
data.iloc[:, :3][data.three > 5]

Boolean arrays can be used with loc but not iloc:

In [ ]:
data.loc[data.three >= 2]

### 2.3 Combining and Merging Datasets

Data contained in pandas objects can be combined in a number of ways:

`pandas.merge`
Connect rows in DataFrames based on one or more keys. This will be familiar to users of SQL or other relational databases, as it implements database `join` operations.

`pandas.concat`
Concatenate or “stack” objects together along an axis.

`combine_first`
Splice together overlapping data to fill in missing values in one object with values from another.


#### DataFrame joins
Merge or join operations combine datasets by linking rows using one or more keys. These operations are particularly important in relational databases (e.g., SQL-based).

In [ ]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})

In [ ]:
df1

In [ ]:
df2

In [ ]:
pd.merge(df1, df2)

Note that I didn’t specify which column to join on. If that information is not specified, pandas.merge uses the overlapping column names as the keys. It’s a good practice to specify explicitly, though:

In [ ]:
pd.merge(df1, df2, on="key")

In general, the order of column output in pandas.merge operations is unspecified. If the column names are different in each object, you can specify them separately:

In [ ]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})
df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})

In [ ]:
df3

In [ ]:
df4

In [ ]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

The "c" and "d" values and associated data are missing from the result. By default, pandas.merge does an "inner" join; the keys in the result are the intersection, or the common set found in both tables. Other possible options are "left", "right", and "outer". The outer join takes the union of the keys, combining the effect of applying both left and right joins:

In [ ]:
pd.merge(df1, df2, how="outer")

In [ ]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

In an outer join, rows from the left or right DataFrame objects that do not match on keys in the other DataFrame will appear with NA values in the other DataFrame’s columns for the nonmatching rows.



Many-to-many left and inner joins examples:



In [ ]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                    "data1": pd.Series(range(6), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"],
                    "data2": pd.Series(range(5), dtype="Int64")})

In [ ]:
df1

## 

In [ ]:
df2

In [ ]:
pd.merge(df1, df2, on="key", how="left")

In [ ]:
pd.merge(df1, df2, how="inner")

In some cases, the merge key(s) in a DataFrame will be found in its index (row labels). In this case, you can pass `left_index=True` or `right_index=True` (or both) to indicate that the index should be used as the merge key:

In [ ]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])


In [ ]:
left1

In [ ]:
right1

In [ ]:
pd.merge(left1, right1, left_on="key", right_index=True)

#### Concatenating

Another kind of data combination operation is referred to interchangeably as concatenation or stacking. NumPy’s concatenate function can do this with NumPy arrays:

In [ ]:
arr = np.arange(12).reshape((3, 4))


In [ ]:
arr

In [ ]:
np.concatenate([arr, arr], axis=1)

The concat function in pandas provides a consistent way to combine Series or DataFrames taking into account index information. Suppose we have three Series with no index overlap:



In [ ]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")

In [ ]:
s1

In [ ]:
s2

In [ ]:
s3

In [ ]:
pd.concat([s1, s2, s3])

In [ ]:
pd.concat([s1, s2, s3], axis="columns")


By default, pandas.concat works along `axis="index"`, producing another Series. If you pass `axis="columns"`, the result will instead be a DataFrame:

In [ ]:
pd.concat([s1, s2, s3], axis="columns")

In this case there is no overlap on the other axis, which as you can see is the union (the "outer" join) of the indexes. You can instead intersect them by passing `join="inner"`:

In [ ]:
s4 = pd.concat([s1, s3])

In [ ]:
s4

In [ ]:
pd.concat([s1, s4], axis="columns")

In [ ]:
pd.concat([s1, s4], axis="columns", join="inner")

## 3. Extra Reading

### 3.1 Web Scraping with Selenium
#### Suggested Reading: https://www.scrapingbee.com/blog/selenium-python/

### 3.2 Read files to DataFrames
#### Suggested Reading: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

### 3.3 Reshape and Pivoting
#### Suggested Reading: https://pandas.pydata.org/docs/user_guide/reshaping.html

### References:
* https://www.freecodecamp.org/news/web-scraping-sci-fi-movies-from-imdb-with-python/
* https://wesmckinney.com/book/